# Praten met LLMs

Iedereen weet hoe ze moeten praten met LLMs als ChatGPT, Gemini, Claude, etc.: via een website. Dat is voor individueel gebruik natuurlijk perfect, maar wat als je een LLM voor een eigen project in wil zetten? Dan kan dat niet via de browser, dit doen we met een API. Hieronder heb ik een voorbeeldje om te praten met een model van [Nebius Studio](https://tokenfactory.nebius.com/). Dit is een platform waar je gratis 1 euro krijgt om met LLMs te praten. Dit is waarschijnlijk niet genoeg voor je project, maar voor kleine tests werkt dat uitstekend.

Eerst de imports dan maar: Nebius werkt op de zelfde manier als Open AI, dit is een soort standaard geworden voor vele LLM API aanbieders

In [ ]:
# pip install openai

In [ ]:
import os
from openai import OpenAI

Als we werken met een API hebben we een *key* nodig. Dit is een verificatiesleutel die je stuurt naar de API waarmee je laat weten dat jij het bent.

Ik heb op de nebius studio website een key aangemaakt en toegevoegd aan mijn windows systeem via de volgende stappen:
- Press Start -> type “Environment Variables” -> open “Edit the system environment variables”
- Click Environment Variables
- Under User variables (or System variables):
  - Click New
  - Variable name: HCAI_NEBIUS_API_KEY
  - Variable value: your key
- Press OK -> restart your notebook\
*(Steps generated with ChatGPT)*

<span style="color: red; font-weight: bold;">LET OP: Je wil NOOIT je API key delen. Hiermee kunnen anderen mensen jou geld uitgeven</span>   


<span style="color: red; font-weight: bold;">Push de key dus nooit naar GitHub. Ook niet als je deze ergens in een jupyter notebook hebt gebruikt. Dit valt allemaal terug te halen. Reset dan eerst je notebook en zet deze daarna pas in Git.</span>   

In [ ]:
NEBIUS_API_KEY = os.environ.get("HCAI_NEBIUS_API_KEY") # <- Ik heb mijn API key `HCAI_NEBIUS_API_KEY` genoemd, als die van jou anders heet, pas deze dan aan

if not NEBIUS_API_KEY:
    raise ValueError("NEBIUS_API_KEY is not set in environment variables.")

Nu kunnen we contact maken met de API. Hiervoor maken we een client aan. De client is onze verbinding met de API. We hoeven hier nog niet ons model te specificeren, dat komt straks pas.

In [ ]:
client = OpenAI(
    api_key=NEBIUS_API_KEY,
    base_url="https://api.studio.nebius.ai/v1",
)

Oke, nu kunnen we een functie gaan schrijven die ene vraag stelt aan het model en het antwoord teruggeeft. Hieronder een voorbeeld:

In [ ]:
def ask_nebius(model: str, prompt: str)-> tuple[object, str, str]:
    """
    Sends a user message to a Nebius Studio LLM and returns the model's reply.
    """
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "user", "content": prompt} 
        ],
    )
    response = client.chat.completions.create(
        model=model,
        messages=[ # Hier geven we aan hoe het model de prompt van de user moet ontvangen.
            # hier kunnen we een system prompt meegeven. 
            # Dit kan bijvoorbeeld een instructie zijn voor hoe het model zich moet gedragen tegenover de gebruiker.
            {
                "role": "system",
                "content": """SYSTEM_PROMPT""" 
            },
            # Hier geven we de user prompt.
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": prompt
                    }
                ]
            }
        ]
    )
    
    # De LLM geeft veel informatie terug, maar we zijn alleen geïnteresseerd in de response tekst. 
    # We geven eerst alle output, dan alle output als een json format, en daarna alleen de message response.
    return response, response.to_json(), response.choices[0].message.content

Oke, laten we een simpele vraag stellen:

In [ ]:
# Op dit moment is dit model het goedkoopst
model = "google/gemma-2-2b-it"
response, json_response, text_reply = ask_nebius(model, "Give me the python code for a hello world example.")

print("Model reply:\n", text_reply)


Hieronder heb ik de output die ik zelf kreeg geplakt in markdown, zodat de opmaak klopt:

---

Model reply:
 ```python
print("Hello, world!")
```

**Explanation:**

* **`print()`:** This is a built-in Python function that displays whatever you put inside its parentheses on the screen.
* **"Hello, world!"** : This is a string, a sequence of characters enclosed within quotation marks.  This is the message you want to display.


**To run this code:**

1. **Save it:** Create a new file named `hello_world.py` and paste this code into it.
2. **Open a terminal:** Navigate to the directory containing the file using the `cd` command.
3. **Run the code:** Type `python hello_world.py` and press Enter.


You'll see "Hello, world!" printed in your terminal. 

